In [ ]:
# Load Whisper
import whisper
print("Loading Whisper model...")
model = whisper.load_model("medium", device="cpu")
print("Model loaded!")

# FastAPI imports
import nest_asyncio, uvicorn
from fastapi import FastAPI, File, UploadFile
from fastapi.responses import JSONResponse
from fastapi.middleware.cors import CORSMiddleware
import tempfile
from pydub import AudioSegment

nest_asyncio.apply()

app = FastAPI()
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_methods=["*"],
    allow_headers=["*"],
)

@app.post("/transcribe")
async def transcribe_audio(file: UploadFile = File(...)):
    try:
        suffix = file.filename.split(".")[-1] or ".webm"
        with tempfile.NamedTemporaryFile(delete=False, suffix=suffix) as tmp:
            tmp.write(await file.read())
            tmp_path = tmp.name

        # Convert to wav if needed
        if suffix.lower() != "wav":
            wav_path = tmp_path.rsplit(".", 1)[0] + ".wav"
            audio = AudioSegment.from_file(tmp_path)
            audio.export(wav_path, format="wav")
            tmp_path = wav_path

        result = model.transcribe(tmp_path, language="es", verbose=False)
        import os; os.remove(tmp_path)

        return JSONResponse({"text": result.get("text", "").strip()})
    except Exception as e:
        return JSONResponse({"error": str(e)}, status_code=500)

uvicorn.run(app, host="0.0.0.0", port=8000)


In [1]:
# ------------------------------
# FastAPI Whisper Spanish-only API
# ------------------------------

import whisper
import nest_asyncio
import uvicorn
from fastapi import FastAPI, File, UploadFile
from fastapi.responses import JSONResponse
from fastapi.middleware.cors import CORSMiddleware
import tempfile
from pydub import AudioSegment
import os

# Allow running in Jupyter / notebooks
nest_asyncio.apply()

# ---------------- Settings ----------------
LANGUAGE = "es"          # Force Spanish
MODEL_SIZE = "medium"
DEVICE = "cpu"
SAMPLE_RATE = 44100      # Force 44.1 kHz

print("Loading Whisper model...")
model = whisper.load_model(MODEL_SIZE, device=DEVICE)
print("Model loaded!")

# ---------------- FastAPI ----------------
app = FastAPI()
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_methods=["*"],
    allow_headers=["*"],
)

@app.post("/transcribe")
async def transcribe_audio(file: UploadFile = File(...)):
    try:
        # Get file suffix
        suffix = "." + file.filename.split(".")[-1] if "." in file.filename else ".webm"

        # Save uploaded file temporarily
        with tempfile.NamedTemporaryFile(delete=False, suffix=suffix) as tmp:
            tmp.write(await file.read())
            tmp_path = tmp.name

        # Convert to WAV mono 44.1 kHz if needed
        if suffix.lower() != ".wav":
            wav_path = tmp_path.rsplit(".", 1)[0] + ".wav"
            audio = AudioSegment.from_file(tmp_path).set_channels(1).set_frame_rate(SAMPLE_RATE)
            audio.export(wav_path, format="wav")
            tmp_path = wav_path

        # Transcribe forcing Spanish
        result = model.transcribe(tmp_path, language=LANGUAGE, task="transcribe", verbose=False)

        # Clean up temporary file
        try:
            os.remove(tmp_path)
        except:
            pass

        return JSONResponse({"text": result.get("text", "").strip()})

    except Exception as e:
        return JSONResponse({"error": str(e)}, status_code=500)

# ---------------- Run Server ----------------
if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)

/home/mlw19mlw91/Downloads/.venv/lib/python3.12/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
100%|██████████| 48/48 [00:04<00:00,  9.68frames/s]

INFO:     192.168.1.148:33704 - "POST /transcribe HTTP/1.1" 200 OK



/home/mlw19mlw91/Downloads/.venv/lib/python3.12/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
100%|██████████| 48/48 [00:04<00:00, 10.81frames/s]

INFO:     192.168.1.148:33714 - "POST /transcribe HTTP/1.1" 200 OK



/home/mlw19mlw91/Downloads/.venv/lib/python3.12/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
100%|██████████| 48/48 [00:05<00:00,  9.57frames/s]

INFO:     192.168.1.148:47526 - "POST /transcribe HTTP/1.1" 200 OK



/home/mlw19mlw91/Downloads/.venv/lib/python3.12/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
100%|██████████| 48/48 [00:18<00:00,  2.55frames/s]

INFO:     192.168.1.148:47538 - "POST /transcribe HTTP/1.1" 200 OK



INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [9591]
